In [1]:
import scipy
import numpy as np
import pandas as pd
from scipy import special
from scipy.optimize import minimize
from scipy.interpolate import interp1d

from mpmath import invertlaplace
from mpmath import *
mp.dps = 10; mp.pretty = True

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_2D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta = np.arctan2(x[1], x[0])   
    
    return ndt+rt, theta

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
    term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
    term3 = da(t) - (a(t)/(t-tau)) + kk
    term4 = special.iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    term5 = (np.sqrt(a(t)*z)/(t-tau)) * special.iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q)

        for j in range(1, n):
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_2D_likelihood(prms, RT, Theta):
    ndt = prms[1]
    mu = np.array([prms[2], prms[3]])
    
    if prms[0] <= 2:
        a = lambda t: prms[0]**2
        da = lambda t: 0
        fpt = ie_bessel_fpt(a, da, mu.shape[0], 0.001, 
                            dt=0.01, T_max=max(RT))
    else:
        a = prms[0]
        nu = (mu.shape[0]-2)/2
        fixed = 2**nu * gamma(1+nu)
        fpt = lambda p: (a*sqrt(2*p))**nu/fixed *  1./besseli(nu, a*sqrt(2*p))
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001:
            mu_dot_x0 = mu[0]*np.cos(theta)
            mu_dot_x1 = mu[1]*np.sin(theta)
            term1 = prms[0] * (mu_dot_x0 + mu_dot_x1)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt - ndt)
            
            if prms[0] < 2: 
                density = exp(term1 - term2) * fpt(rt - ndt)

                if 0.1**14 < density:
                    log_lik += -log(density)
                else:
                    log_lik += -log(0.1**14)
            else:
                density = exp(term1 - term2) * invertlaplace(fpt, rt-ndt, method='talbot')
                if 0.1**14 < density:
                    log_lik += -log(density)
                else:
                    log_lik += -log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': []}

min_threshold = 0.5
max_threshold = 5

min_ndt = 0.1
max_ndt = 1

min_mu = -3.5
max_mu = 3.5

In [6]:
for n in range(3):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    
    RT = []
    Theta = []
    
    print(n+1, end=': ')
    for i in range(250):
        rt, theta = simulate_HSDM_2D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    print('Simulation is done!', end=' / ')
    
    min_ans = minimize(HSDM_2D_likelihood,
                       args=(RT, Theta), 
                       x0=np.array([np.random.uniform(min_threshold, max_threshold),
                                    np.random.uniform(min_ndt, max_ndt), 
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu)]),
                       method='Nelder-Mead', 
                       bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                               (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['mu1_estimate'].append(min_ans.x[2])
    recovery_df['mu2_estimate'].append(min_ans.x[3])
    
    print('Estimation is done!')
    
recovery_df = pd.DataFrame(recovery_df)

1: Simulation is done! / Estimation is done!
2: Simulation is done! / Estimation is done!
3: Simulation is done! / Estimation is done!


In [7]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
0,1.123020,0.500000,0.377408,0.450408,-2.788457,-1.700494,-1.963852,-3.500000
1,2.097202,1.817862,0.916138,1.000000,3.273374,3.500000,2.835600,3.500000
2,1.173229,1.159561,0.296946,0.301825,1.689765,1.707268,-2.379505,-2.333999


In [8]:
recovery_df.corr()

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
threshold_true,1.000000,0.887748,0.986326,0.969114,0.733849,0.792909,0.993099,0.993923
threshold_estimate,0.887748,1.000000,0.799742,0.746806,0.964181,0.984401,0.827635,0.933024
ndt_true,0.986326,0.799742,1.000000,0.996506,0.611857,0.681642,0.998847,0.962191
ndt_estimate,0.969114,0.746806,0.996506,1.000000,0.543657,0.618149,0.991349,0.936080
mu1_true,0.733849,0.964181,0.611857,0.543657,1.000000,0.995808,0.649116,0.804164
mu1_estimate,0.792909,0.984401,0.681642,0.618149,0.995808,1.000000,0.715975,0.855163
mu2_true,0.993099,0.827635,0.998847,0.991349,0.649116,0.715975,1.000000,0.974155
mu2_estimate,0.993923,0.933024,0.962191,0.936080,0.804164,0.855163,0.974155,1.000000


In [9]:
recovery_df.to_csv('Hybrid_2d_recovery.csv')